# Find all valuesets in differential of StructureDefinitions

- for hard coded path search all SD for binding and list the valuesets
- list em in a md table format in std out

In [ ]:
from pathlib import Path
from json import load, dumps, loads
from yaml import load as y_load, dump as y_dump, FullLoader
from fhir.resources import construct_fhir_element
from requests import get

headers = {'Accept': 'application/json+fhir',}
auth = ('apikey','5a96c64e-2156-43c0-a9b3-1c2c8d914bb5')
my_path = Path(r'/Users/ehaas/Documents/FHIR/US-Core/output')
data_path = Path(r'/Users/ehaas/Documents/FHIR/US-Core/input/data')
uscore_package_path = Path(r'/Users/ehaas/.fhir/packages/hl7.fhir.us.core#dev')
fhir_package_path = Path(r'/Users/ehaas/.fhir/packages/hl7.fhir.r4.core#4.0.1')
my_path, uscore_package_path, fhir_package_path

In [ ]:
my_valuesets = set()
for i in my_path.glob('Struct*.json'):
    # print(i)
    obj = load(i.open())
    # print(obj['id'])
    #find reference type elements in diff
    for diff in obj['differential']['element']:
        # print(f"diff['id']= {diff['id']}")
        for snapshot in obj['snapshot']['element']:
            if snapshot['id'] == diff['id']:
                try:
                    # print(f'snapshot["binding"]["valueSet"] = {snapshot["binding"]["valueSet"]}')
                    my_valuesets.add(snapshot["binding"]["valueSet"])

                except KeyError:
                    # print(f"key error")
                    pass

my_valuesets


### get codesystems by looking up valuesets  package or VSAC


In [22]:
my_codesystems = set()

def get_vs_obj(vs_url,vs_filename):
    try:
        vs_file = fhir_package_path / 'package' / vs_filename
        vs_obj = loads(vs_file.read_text())
    except:
        try:
            vs_file = uscore_package_path / 'package' / vs_filename
            vs_obj = loads(vs_file.read_text())
        except:
            # print("-----------------")
            # print(vs)
            vs_obj = get(vs_url, headers=headers, auth=auth).json()
    return vs_obj

def get_system(vs_obj):
    for cs in vs_obj['compose']['include']:
        try:
            my_codesystems.add(cs['system'])
        except KeyError:
            # print(vs_obj['id'])
            # print("==============")
            # print(cs['valueSet'])
            for vs in cs['valueSet']:
                vs_filename = f"ValueSet-{vs.split('/')[-1].split('|')[0]}.json"
                # print(vs_filename)
                new_vs_obj = get_vs_obj(vs,vs_filename)
                # print("++++++++++++++++++++++")
                # print(new_vs_obj['id'], new_vs_obj['compose']['include'])
                get_system(new_vs_obj)

for i,vs in enumerate(my_valuesets):
    # print(vs)
    vs_filename = f"ValueSet-{vs.split('/')[-1].split('|')[0]}.json"
    vs_obj = get_vs_obj(vs,vs_filename)
    get_system(vs_obj)


my_codesystems

{'http://hl7.org/fhir/CodeSystem/medicationrequest-intent',
 'http://hl7.org/fhir/CodeSystem/medicationrequest-status',
 'http://hl7.org/fhir/address-use',
 'http://hl7.org/fhir/administrative-gender',
 'http://hl7.org/fhir/care-team-status',
 'http://hl7.org/fhir/contact-point-system',
 'http://hl7.org/fhir/contact-point-use',
 'http://hl7.org/fhir/diagnostic-report-status',
 'http://hl7.org/fhir/document-reference-status',
 'http://hl7.org/fhir/encounter-status',
 'http://hl7.org/fhir/event-status',
 'http://hl7.org/fhir/fm-status',
 'http://hl7.org/fhir/goal-status',
 'http://hl7.org/fhir/location-status',
 'http://hl7.org/fhir/name-use',
 'http://hl7.org/fhir/narrative-status',
 'http://hl7.org/fhir/observation-status',
 'http://hl7.org/fhir/questionnaire-answers-status',
 'http://hl7.org/fhir/request-intent',
 'http://hl7.org/fhir/request-status',
 'http://hl7.org/fhir/sid/cvx',
 'http://hl7.org/fhir/sid/icd-10-cm',
 'http://hl7.org/fhir/sid/icd-9-cm',
 'http://hl7.org/fhir/us/cor

### add resolvable links
- right now only need for the http://hl7.org/fhir/ ones
  - use the hl7 package other/spec_internals file (~/.fhir/packages/hl7.terminology.r4#5.0.0/package/other/spec.internals)
- rest can copy the url
- unless a urn then link is None

In [33]:
spec_internals = fhir_package_path / 'package' / 'other' / 'spec.internals'
si=loads(spec_internals.read_text(encoding='utf-8-sig'))
paths = si['paths']
paths
cs_dict = {}
for cs in sorted(my_codesystems):
    try:
        if cs.startswith('http://hl7.org/fhir'):
            try:
              cs_dict[cs] = (v for k,v in paths.items() if cs == k).__next__()
            except StopIteration:
                cs_dict[cs] = cs
        else:
            cs_dict[cs] = cs

cs_dict

SyntaxError: expected 'except' or 'finally' block (1011016194.py, line 16)

### Write to file

In [ ]:
intro_comment = '''
# This file was generated by the binding_finder.ipynb notebook.
# It contains a list of all the codesystems used in the US-Core IG.
# This file is used by the IG publisher to generate the terminology.html#code-system-uris-used-in-us-core section. 
# ANY EDITS YOU MAKE TO THIS FILE WILL BE OVERWRITTEN.
# If you want to add a codesystem to the list, add it to the binding_finder.ipynb notebook
'''
out = data_path / 'codesystems.yml'
print( f'writing file to {out}')
print(f'{intro_comment}\n\n{y_dump(sorted(my_codesystems), default_flow_style=False, sort_keys=False)}')
# out.write_text(f'{intro-comment}\n\n{y_dump(sorted(my_codesystems), default_flow_style=False, sort_keys=False)}')
